# Подготовка данных

Скачать файл можно по ссылке [nutrition5k_dataset.tar.gz (181.4 GB)](https://github.com/google-research-datasets/Nutrition5k)

Содержимое файла: 
- id блюд в формате txt (с разбивкой на train и test)
- RGB фотографии блюд (одно фото на блюдо), карты глубины
- видео блюд с четырех камер 
- csv файл с КБЖУ блюд

Извлекаем все файлы кроме видео

In [11]:
import ffmpeg
import tarfile
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
def x_files(members):
    for tarinfo in members:
        if os.path.splitext(tarinfo.name)[1] == ".txt" or \
            os.path.splitext(tarinfo.name)[1] == ".csv" or \
            os.path.splitext(tarinfo.name)[1] == ".py" or \
            os.path.splitext(tarinfo.name)[1] == ".sh" or \
            os.path.splitext(tarinfo.name)[1] == ".png":
            yield tarinfo
            

# tar = tarfile.open('nutrition5k_dataset.tar', 'r:gz')
# tar.extractall(members=x_files(tar))
# tar.close()


Попробуем достать каждый пятый кадр в каждом видео (для каждого блюда представлено 4 видео с разных камер)

In [3]:
tar = tarfile.open('nutrition5k_dataset.tar', 'r:gz')

In [4]:
list_of_file_names = tar.getmembers()

In [8]:
file_names = [list_of_file_names[i].name for i in range(len(list_of_file_names))]
len(file_names)

37478

In [20]:
for i in tqdm(range(0, len(file_names))):
    if '.h264' in file_names[i]:
        url = file_names[i]
        tar.extract(list_of_file_names[i])
        probe = ffmpeg.probe(filename=url)
        width = probe['streams'][0]['width']
        try:
            ffmpeg.input(url).filter('scale', width, -1).output(url.split('.')[0]+'_frame_%03d.jpeg').run()
            cur_dir = '/'.join(url.split('/')[:-1])
            arr = os.listdir(cur_dir)
            for i in arr:
                if i.split('.')[0][-1] != '5':
                    os.remove(cur_dir+'/' + i)
        except:
            print('error:', i, file_names[i])
        
        
        

C:\Users\Yulia\AppData\Local\Temp\ipykernel_14296\969829498.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(15386, len(file_names))):


  0%|          | 0/22092 [00:00<?, ?it/s]

error: 15386 nutrition5k_dataset/imagery/side_angles/dish_1560368815/camera_D.h264
